In [ ]:
from data_module_F import *
from feature_module_F import *

from model_module_F import *
from evaluation_module_F import *
from EfficientNet1D_builder import *
from model_builder import *

In [2]:
####### The 5-folds for cross validation are stored in an external folder 'dataset' in order to run the models simultaneously ###

path = 'dataset/' ## data path
exp = 'test' ## experiment name 
seed = 1 ### the number of fold (from 1 to 5)
route = './models/'####Inset ## experiments folder 

epochs = 300
lr = 0.0005
kind = 'efficientnet' ## type of model architecture other options can be None: simple CNN or 'resnet'

GP = False ## True to run gap filling from CNNIncomplete

dir_n = route + '{}_{}/'.format(exp,seed) ## Specific experiment dir to be created
create_path(dir_n)

tf.random.set_seed(155) ### seed for the experiement

In [ ]:
######## GPU RAM memory if GPU available ##########
os.environ["CUDA_VISIBLE_DEVICES"]="1"

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 15*1GB of memory on the first GPU
    try:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=1024*5)])
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
            print(e)

### Load data 

In [4]:
# ## Balanced number of samples ### 
db_train, X_train, y_train = read_db(path + 'fillCV_{}.csv'.format(seed),sp=True)
db_test, X_test, y_test = read_db(path + 'testCV_{}.csv'.format(seed),sp=True)

gap_fil = db_train.copy()

samp_w_tr = pd.read_csv(path + 'samp_w_tr_{}.csv'.format(seed)).drop(['Unnamed: 0'],axis=1).loc[:,'0']

In [5]:
Traits = y_train.columns.to_list()
Traits

['Anthocyanin content (μg/cm²)',
 'Boron content (mg/cm²)',
 'C content (mg/cm²)',
 'Ca content (mg/cm²)',
 'Carotenoid content (μg/cm²)',
 'Cellulose (mg/cm²)',
 'Chl content (μg/cm²)',
 'Copper content (mg/cm²)',
 'EWT (mg/cm²)',
 'Fiber (mg/cm²)',
 'LAI (m²/m²)',
 'LMA (g/m²)',
 'Lignin (mg/cm²)',
 'Magnesium content (mg/cm²)',
 'Manganese content (mg/cm²)',
 'N content (mg/cm²)',
 'NSC (mg/cm²)',
 'Phosphorus content (mg/cm²)',
 'Potassium content (mg/cm²)',
 'Sulfur content (mg/cm²)']

## CNN Incomplete ############   

In [6]:
#### Final global model training####

path_g = dir_n + 'incomplete/'
create_path(path_g)

######### Train set ######
# train_x, train_y, samp_w_tr = data_prep('400', gap_fil, Traits, w_train = gap_fil.loc[:,:'Site'], multi= True)
train_x, train_y= data_prep('400', gap_fil, Traits, multi= True)
# samp_w_tr = None

    ##############
scaler_list = save_scaler(train_y, save=True, dir_n = path_g, k = 'global')


val_x = train_x.sample(frac = 0.1,random_state = seed)
val_y = train_y.loc[val_x.index,:]
samp_w_val = pd.DataFrame(samp_w_tr).sample(frac = 0.1,random_state = seed)

if(samp_w_tr is not None):
    if (samp_w_tr.sum().sum() !=0):    
        train_ds = dataset(train_x.drop(val_x.index), train_y.drop(val_y.index), pd.DataFrame(samp_w_tr).drop(samp_w_val.index), scaler_list, Traits, shuffle=True,augment=True)
        test_ds = dataset(val_x, val_y, samp_w_val, scaler_list, Traits)
else:
    train_ds = dataset(train_x.drop(val_x.index), train_y.drop(val_y.index), None, scaler_list, Traits, shuffle=True,augment=True)
    test_ds = dataset(val_x, val_y, None, scaler_list, Traits)

    ##### Model definition  and taraining #######
input_shape = train_x.shape[1]
output_shape = train_y.shape[1]

In [7]:
EPOCHS = epochs 
# EPOCHS = 2

######model definition ####
best_model = model_definition(input_shape, output_shape,lr = lr, kind= kind)

checkpoint_path = path_g + 'checkpoint'
create_path(checkpoint_path)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
filepath = checkpoint_path + "/epoch{epoch:02d}-val_root_mean_squared_error{val_root_mean_squared_error:.2f}.hdf5",
save_weights_only=True,
monitor = 'val_root_mean_squared_error',
mode='min',
save_best_only=True)

######## Model training#######
his = best_model.fit(train_ds,
                validation_data = test_ds,
                epochs = EPOCHS,
                verbose=1, callbacks = [model_checkpoint_callback])

val_acc_per_epoch = his.history['val_root_mean_squared_error']
best_epoch = val_acc_per_epoch.index(min(val_acc_per_epoch)) + 1    

########### Save best model ########
path_trial = path_g + "Model.json"
path_best = checkpoint_path + "/epoch{0:02d}-val_root_mean_squared_error{1:.2f}.hdf5".format(best_epoch,min(val_acc_per_epoch))
path_w = path_g + 'Trial_weights.h5'
save_model(best_model, path_trial, path_best, path_w)

Epoch 1/2


2023-03-09 13:24:11.443511: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8100
2023-03-09 13:24:12.178497: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-09 13:24:12.179553: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-09 13:24:12.179566: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-03-09 13:24:12.179807: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-03-09 13:24:12.179840: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-03-09 13:24:13.700072: I tensorflow/stream_executor/cuda/c

686/686 [==============================] - 17s 16ms/step - loss: 38.6367 - root_mean_squared_error: 1.2006 - r_square: -0.4453 - val_loss: 29.5776 - val_root_mean_squared_error: 0.8372 - val_r_square: 0.3142
Epoch 2/2
686/686 [==============================] - 11s 15ms/step - loss: 27.8990 - root_mean_squared_error: 0.8086 - r_square: 0.3444 - val_loss: 25.5665 - val_root_mean_squared_error: 0.7745 - val_r_square: 0.4132


In [8]:
############## Evaluation ###########
# test_x, test_y, samp_w_test = data_prep('400', db_test, Traits, w_train = w_test, multi = True)
test_x, test_y = data_prep('400', db_test, Traits, multi = True)

pred = scaler_list.inverse_transform(best_model.predict(test_x))
pred_df = pd.DataFrame(pred, columns = test_y.columns+ ' Predictions')
# pred_df.to_csv(path_g + 'Predictions.csv')

obs_pf = pd.DataFrame(test_y)
# obs_pf.to_csv(path_g + 'Observations.csv')

# test = all_scores(Traits,Traits,obs_pf, pred_df,samp_w_test)
test = all_scores(Traits,Traits,obs_pf, pred_df,None)
test
# test.to_csv(path_g + 'Global_all.csv')

,Anthocyanin content (μg/cm²),Boron content (mg/cm²),C content (mg/cm²),Ca content (mg/cm²),Carotenoid content (μg/cm²),Cellulose (mg/cm²),Chl content (μg/cm²),Copper content (mg/cm²),EWT (mg/cm²),Fiber (mg/cm²),LAI (m²/m²),LMA (g/m²),Lignin (mg/cm²),Magnesium content (mg/cm²),Manganese content (mg/cm²),N content (mg/cm²),NSC (mg/cm²),Phosphorus content (mg/cm²),Potassium content (mg/cm²),Sulfur content (mg/cm²)
r2_score,-0.049093,0.427159,0.717542,0.229613,0.084339,0.675160,0.052550,0.471669,0.226677,0.706488,0.120971,0.695001,0.662278,0.263813,0.389777,0.317377,0.610052,0.435022,0.150556,0.453420
RMSE,0.448614,0.000168,2.368249,0.088222,2.960750,1.065992,16.102486,0.000024,7.891765,2.478216,1.502386,37.476866,1.406804,0.013102,0.001823,0.076744,1.839519,0.006858,0.056195,0.007082
nRMSE (%),22.559681,14.367622,10.421544,19.405616,19.668824,13.002392,26.144361,14.421728,17.944483,12.829179,23.152776,10.389548,13.689166,16.580550,17.296541,16.011354,12.703782,16.074477,20.520606,13.687594
MAE,0.343804,0.000119,1.562723,0.054015,2.034171,0.754831,11.414880,0.000018,5.190689,1.762771,1.208988,25.303953,0.977716,0.009109,0.001367,0.055878,1.185594,0.005014,0.039738,0.005188
Bias,0.114984,-0.000002,-0.107896,0.015902,-0.171460,0.043216,-3.528112,-0.000003,2.131168,0.002895,-0.016249,0.395456,0.059497,0.000820,-0.000101,-0.009709,0.054322,0.000326,0.013654,-0.000109


## CNN Inexact ############   

In [9]:
GP = True ## True to run gap filling from CNNIncomplete
path_g

'./models/test_1/incomplete/'

In [10]:
if (GP):
    for t in range(len(Traits)):
        gap_fil = fill_gp(gap_fil, best_model, scaler_list, Traits, t)
    gap_fil.to_csv(path_g + 'Gapfil_allTraits.csv')

    #############Inexact ############   

    mx = pd.concat([db_train[Traits].quantile(0.99),db_test[Traits].quantile(0.99)],axis=1).T.max()
    s = (gap_fil.reset_index(drop=True)[Traits]>mx)
    idx = np.where(s)[0]

    mx = pd.concat([db_train[Traits].quantile(0.01),db_test[Traits].quantile(0.01)],axis=1).T.min()
    s = (gap_fil.reset_index(drop=True)[Traits]<mx)
    idx1 = np.where(s)[0]

    gap_fil = gap_fil.reset_index(drop=True).drop(list(idx) + list(idx1),axis=0)


    #### Final global model training
    path_g = dir_n + 'inexact/'
    create_path(path_g)

    ######### Train set ######

    # train_x, train_y, samp_w_tr = data_prep('400', gap_fil, Traits, w_train = gap_fil.loc[:,:'Site'], multi= True)
    train_x, train_y = data_prep('400', gap_fil, Traits, multi= True)
    samp_w_tr = samp_w_tr.loc[train_x.index]

        ##############
    scaler_list = save_scaler(train_y, save=True, dir_n = path_g, k = 'global')

    val_x = train_x.sample(frac = 0.1,random_state = seed)
    val_y = train_y.loc[val_x.index,:]
    samp_w_val = pd.DataFrame(samp_w_tr).sample(frac = 0.1,random_state = seed)

    if(samp_w_tr is not None):
        if (samp_w_tr.sum().sum() !=0):    
            train_ds = dataset(train_x.drop(val_x.index), train_y.drop(val_y.index), pd.DataFrame(samp_w_tr).drop(samp_w_val.index), scaler_list, Traits, shuffle=True,augment=True)
            test_ds = dataset(val_x, val_y, samp_w_val, scaler_list, Traits)
    else:
        train_ds = dataset(train_x.drop(val_x.index), train_y.drop(val_y.index), None, scaler_list, Traits, shuffle=True,augment=True)
        test_ds = dataset(val_x, val_y, None, scaler_list, Traits)

        ##### Model definition  and taraining #######
    input_shape = train_x.shape[1]
    output_shape = train_y.shape[1]

    EPOCHS = epochs 
    # EPOCHS = 2 
    best_model = model_definition(input_shape, output_shape,lr = lr, kind= kind)

    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,restore_best_weights=True)

    checkpoint_path = path_g + 'checkpoint'
    create_path(checkpoint_path)

    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path + "/epoch{epoch:02d}-val_root_mean_squared_error{val_root_mean_squared_error:.2f}.hdf5",
    save_weights_only=True,
    monitor = 'val_root_mean_squared_error',
    mode='min',
    save_best_only=True)

    his = best_model.fit(train_ds,
                    validation_data = test_ds,
                    epochs = EPOCHS,
                    verbose=1, callbacks = [model_checkpoint_callback])

    val_acc_per_epoch = his.history['val_root_mean_squared_error']
    best_epoch = val_acc_per_epoch.index(min(val_acc_per_epoch)) + 1    

    path_trial = path_g + "Model.json"
    path_best = checkpoint_path + "/epoch{0:02d}-val_root_mean_squared_error{1:.2f}.hdf5".format(best_epoch,min(val_acc_per_epoch))
    path_w = path_g + 'Trial_weights.h5'
    save_model(best_model, path_trial, path_best, path_w)

    ####### Evaluation #######
    # test_x, test_y, samp_w_test = data_prep('400', db_test, Traits, w_train = w_test, multi = True)
    test_x, test_y = data_prep('400', db_test, Traits, multi = True)

    pred = scaler_list.inverse_transform(best_model.predict(test_x))
    pred_df = pd.DataFrame(pred, columns = test_y.columns+ ' Predictions')
    # pred_df.to_csv(path_g + 'Predictions.csv')

    obs_pf = pd.DataFrame(test_y)
    # obs_pf.to_csv(path_g + 'Observations.csv')

    test = all_scores(Traits,Traits,obs_pf, pred_df,None)
    # test.to_csv(path_g + 'Global_all.csv')

Epoch 1/3
626/626 [==============================] - 10s 14ms/step - loss: 29.9101 - root_mean_squared_error: 1.1278 - r_square: -0.2690 - val_loss: 18.7908 - val_root_mean_squared_error: 0.6801 - val_r_square: 0.5261
Epoch 2/3
626/626 [==============================] - 9s 14ms/step - loss: 15.4454 - root_mean_squared_error: 0.6196 - r_square: 0.6169 - val_loss: 14.2171 - val_root_mean_squared_error: 0.5893 - val_r_square: 0.6442
Epoch 3/3
626/626 [==============================] - 8s 13ms/step - loss: 14.7923 - root_mean_squared_error: 0.6074 - r_square: 0.6319 - val_loss: 16.8919 - val_root_mean_squared_error: 0.6344 - val_r_square: 0.5876


In [11]:
test

,LMA (g/m²),N content (mg/cm²),LAI (m²/m²),C content (mg/cm²),Chl content (μg/cm²),EWT (mg/cm²),Carotenoid content (μg/cm²),Phosphorus content (mg/cm²),Lignin (mg/cm²),Cellulose (mg/cm²),Fiber (mg/cm²),Anthocyanin content (μg/cm²),NSC (mg/cm²),Magnesium content (mg/cm²),Ca content (mg/cm²),Potassium content (mg/cm²),Boron content (mg/cm²),Copper content (mg/cm²),Sulfur content (mg/cm²),Manganese content (mg/cm²)
r2_score,0.691206,0.397308,0.317797,0.564853,0.347325,-3.122974,0.120619,0.399969,0.525619,-0.291719,0.520178,-0.054464,0.658534,0.192079,0.169353,0.240923,0.344807,0.437615,0.411099,0.353856
RMSE,37.709309,0.072111,1.320485,2.939468,13.364823,18.222127,2.901502,0.007067,1.667314,2.068248,3.168594,0.449761,1.721371,0.013725,0.091607,0.053122,0.000179,0.000024,0.007351,0.001875
nRMSE (%),10.453987,15.044771,20.345358,12.935208,21.699430,41.433908,19.275232,16.565628,16.224109,25.280640,16.403114,22.617356,11.887847,17.369583,20.150287,19.398385,15.365706,14.879248,14.207627,17.798346
MAE,22.863339,0.045093,1.049534,1.619792,7.698539,6.205245,1.933449,0.005025,1.043375,0.909686,1.958032,0.327026,1.021667,0.009288,0.056182,0.037875,0.000119,0.000017,0.004959,0.001246
Bias,6.903955,0.013989,0.080999,0.257489,1.409198,0.472623,0.336126,0.001339,0.371843,0.104295,0.473191,0.181026,0.516975,0.003769,0.024146,0.010490,0.000068,0.000009,0.001520,0.000547
